In [ ]:
# Suppress debugging information about GPU
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import os.path
import sys
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import logging
from matplotlib import pyplot as plt
from numpy import mean, std
from keras.initializers import RandomUniform, GlorotUniform, Zeros
from keras.layers import Dense, InputLayer
from keras.losses import MeanSquaredError
from keras.metrics import BinaryAccuracy
from keras.models import Sequential
from keras.optimizers import *
from keras.regularizers import L2, L1
from keras.callbacks import Callback
from keras.optimizers.schedules import PolynomialDecay, ExponentialDecay
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from cup_helpers import *

# set random seed for reproducible experiments
tf.random.set_seed(42)
keras.utils.set_random_seed(42)

### Utilities

In [ ]:
# read CSV dataset
def read_ds_df(path):
  """
  parse CSV data set and
  returns a tuple (input, target)
  """
  names = ["id", "INPUT_0", "INPUT_1", "INPUT_2", "INPUT_3", "INPUT_4", "INPUT_5", "INPUT_6", "INPUT_7", "INPUT_8", "INPUT_9", "TARGET_x", "TARGET_y", "TARGET_z"]
  data = pd.read_csv(path, dtype=object, delimiter=",", header=None, skiprows=1, names=names)
  y = data.drop(["id","INPUT_0", "INPUT_1", "INPUT_2", "INPUT_3", "INPUT_4", "INPUT_5", "INPUT_6", "INPUT_7", "INPUT_8", "INPUT_9"], axis=1)
  X = data.drop(["id","TARGET_x", "TARGET_y", "TARGET_z"], axis=1).astype(float)
  y = y.astype(float)

  return (X , y)

# read CSV blind test-set
def read_ts_df(path):
  names = ['id','input0', 'input1', 'input2', 'input3', 'input4', 'input5', 'input6', 'input7', 'input8', 'input9']
  df = pd.read_csv(path, names=names, dtype=object, header=None, skipinitialspace=True, skiprows=7)
  return df.drop(['id'],axis=1).astype(float)

In [ ]:
# preprocessing: load and split dataset
(X, y) = read_ds_df("data/ML-CUP23-TR.csv")

#split 80% dev and 20% train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, shuffle=True)

# blind test set
blind_test = read_ts_df("data/ML-CUP23-TS.csv")

In [ ]:
# Mean Euclidean Error metric for sklearn GridSearchCV
def mee(hx, y):

  if y.ndim > 1:
    l2_norms = np.linalg.norm(np.subtract(hx, y), axis=1)
    return mean(l2_norms, axis=0)
  else:
    l2_norms = []
    for p in range(len(y)):
      l2_norms.append(np.linalg.norm(np.subtract(hx[p], y[p])))
    return mean(l2_norms)
      
custom_scores = {
  "mee": make_scorer(mee, greater_is_better=False)
}

In [ ]:
# custom metric to introduce mee on Keras
@tf.autograph.experimental.do_not_convert
def mee_NN(hx, y):
  error = y - hx
  l2_norm = tf.norm(error, ord=2, axis=1)
  return tf.keras.backend.mean(l2_norm, axis=0)

In [ ]:
# get_NN: Function that builds up a NN
def get_NN(X_len, initializer="random", seed=42, hidden_layers=[{"neurons":4,"activation":"tanh"}],
          lr=0.01, alpha=0.5, lambda_reg=None, penalty=None, nesterov=False):

  # weight initialization
  init, bias = (GlorotUniform(seed=seed), Zeros())  if initializer=="glorot" \
          else (RandomUniform(seed=seed), RandomUniform(seed=seed))

  # regularization
  regularizer = L1(l1=lambda_reg) if penalty == "L1" \
          else  L2(l2=lambda_reg) if penalty == "L2" \
          else  None

  # 1 hidden layer
  NN_model = Sequential()
  NN_model.add(InputLayer(input_shape=(X_len,)))

  for layer in hidden_layers:
    NN_model.add(Dense(layer["neurons"],  activation=layer["activation"], 
                kernel_initializer=init, bias_initializer=bias, kernel_regularizer=regularizer)
    )

  # output layer
  NN_model.add(Dense(units=3, activation = "linear", kernel_initializer=init, 
              bias_initializer=bias, kernel_regularizer=regularizer)
  )

  NN_model.compile(optimizer=SGD(learning_rate=lr, momentum=alpha, nesterov=nesterov), loss=mee_NN )
  
  return NN_model

In [ ]:
# utilities to get the mean of K histories

def add_padding(ls, n):
  ls.extend([ls[-1]] * n)
  return ls

def longest(ls):
  return len(max(ls, key=(lambda history : len(history['loss'])))['loss'])

def mean_epochs(l):
  return int(mean([ len(item['loss']) for item in l ]))

def mean_history(_histories):
  m = mean_epochs(_histories)+1
  # m = longest(_histories)
  for history in _histories:
    l = len(history['loss'])
    for field in _histories[0]:
      if l>= m:
        history[field] = history[field][:m]
      else:
        history[field] = add_padding(history[field], (m-l))
  return \
    { field : 
        [ 
          (sum(x)/len(_histories)) for x in zip(
            *[ history[field] for history in _histories ]
          )
        ] for field in _histories[0]
    }

In [ ]:
# static fold counter
def count():
  count.count += 1
  return count.count

def reset_counter():
  count.count =-1

def get_count():
  return count.count

In [ ]:
# static history register
def histories():
  histories.histories = []

def register(h):
  histories.histories.append(h)

def get_histories():
  return histories.histories

def clear_histories():
  histories()

In [ ]:
# plot utility
def do_NN_plot(history):
  plt.plot(history['loss'])
  plt.plot(history['val_loss'],  linestyle="--", color="orange")
  plt.title(f'model MEE')
  plt.ylabel('MEE')
  plt.xlabel('epoch')
  plt.legend(['training', 'test'], loc='upper right')
  plt.show()

In [ ]:
# KerasRegressor Wrapper for kfold
class KRWrapper(KerasRegressor):

  def __init__(self, val_data, k, *args, **kwargs):
    super(KRWrapper, self).__init__(*args, **kwargs)
    self.val_data = val_data
    self.k = k
    
  def fit(self, X, y, **kwargs):
    h = super().fit(X, y, validation_data=self.val_data[count()], **kwargs)
    register(h.history_)
    # do_NN_plot(h.history_)  # plot single fold curve
    if self.kfold_finished(): # plot mean of k folds curves
      do_NN_plot(mean_history(get_histories()))
    
  def kfold_finished(self):
    return self.k == get_count()+1

#### Model Selection

In [ ]:
# lr decay strategies

linear_lr_decay0 = PolynomialDecay(
  0.01,
  200,
  end_learning_rate=0.0005,
  power=1.0,
  name="lr_decay0"
)

linear_lr_decay1 = PolynomialDecay(
  0.01,
  250,
  end_learning_rate=0.0005,
  power=0.5,
  cycle=False,
  name="lr_decay1"
)

linear_lr_decay2 = PolynomialDecay(
  0.01,
  250,
  end_learning_rate=0.0005,
  power=0.5,
  cycle=True,
  name="lr_decay2"
)

exp_lr_decay0 = ExponentialDecay(
  0.2,
  decay_steps=1000,
  decay_rate=0.8,
)

exp_lr_decay1 = ExponentialDecay(
  0.2,
  decay_steps=1000,
  decay_rate=0.2,
)

In [ ]:
# Define grids for gridsearchcv
kerasRegressorParams = {
  "model" : get_NN,
  "X_len" : len(X_train.columns),
  "loss" : mee_NN,
  "optimizer" : "SGD", # fixed into get_NN
  "batch_size" : 32,
  "epochs" : 2000,
  "shuffle" : True,
  "verbose" : 0
}

NN = KerasRegressor(**kerasRegressorParams)

GRID_DICT = {
  # "batch_size" : [8, 25, 32, 64, 128, 500],
  "model__lr" : [linear_lr_decay2], # [0.01, 0.001, 0.005, linear_lr_decay0,linear_lr_decay2, exp_lr_decay1, exp_lr_decay0],
  "model__alpha" : [0.9], # [0.5, 0.6, 0.7, 0.8, 0.85, 0.9],
  "model__initializer" : ["glorot"], # ["random", "glorot"]
  "model__nesterov" : [True], # [True, False]
  "model__penalty": ["L1"], # [None, "L1", "L2"],
  "model__lambda_reg": [0.0005], # [0.01, 0.001, 0.0001, 0.0005],
  "model__seed" : [15],
  "model__hidden_layers" : [  # [ <256, tanh>, <512, tanh>, <128, tanh> ], [ <128, relu>, <256, relu>, <64, relu> ]
    [                         # [ <64, relu>, <128, relu>, <256, relu>, <64, relu> ], [ <64, tanh>, <256, tanh>, <32, tanh> ]
      { 
        "neurons": 128, 
        "activation":"tanh" 
      }, 
      {
        "neurons": 256,
        "activation":"tanh" 
      },
      {
        "neurons": 64,
        "activation":"tanh" 
      }
    ]
  ]
}

grid = GridSearchCV(NN,
                    param_grid=GRID_DICT,
                    scoring=custom_scores,
                    refit="mee",
                    cv=CV,
                    return_train_score=True,
                    n_jobs=-1,
                    
        )

In [ ]:
# exec gridsearch and fit model
grid.fit(X_train, y_train)
print("Best parameters: " + str(grid.best_params_) + " score: " + str(grid.best_score_))

In [ ]:
# print top hyperparameters and results
columns = ['mean_fit_time','param_model__lambda_reg', 'param_model__nesterov','param_model__penalty', 'param_model__alpha', 'param_model__lr', 'mean_test_mee', 'std_test_mee']
top_models = pd.DataFrame(grid.cv_results_).sort_values(by=['mean_test_mee'], ascending=False)[:3]
top_models[columns]

In [ ]:
# second kfold: validation curves, early stopping and mean error of top models

# validation folds
val_split = [ test for (train, test) in CV.split(X_train, y_train) ]

val_data = [ 
  (
    [X_train.iloc[i].tolist() for i in indexes], 
    [y_train.iloc[i].tolist() for i in indexes]
  ) for indexes in val_split 
]

NN_2 = KRWrapper(
  val_data,
  5,
  callbacks=[
    tf.keras.callbacks.EarlyStopping(
      monitor="val_loss", min_delta=0.000001, patience=50, restore_best_weights=True
    )
  ],
  **kerasRegressorParams
)

grid_dict = { "scoring": custom_scores,
              "refit" : False,
              "cv" : CV,
              "return_train_score" : True,
              "n_jobs" : 1,
}

In [ ]:
# exec kfold and select the model

seed = GRID_DICT['model__seed'][0]

n_epochs = []

for params in top_models['params']:
  print(params)

  tr_err, tr_acc, ts_err, ts_acc = [], [], [], []
  for i in range(seed, seed+5):
    print("Seed: " + str(i))

    # reset fold counter and histories
    reset_counter()   
    clear_histories() 

    # set params and seed
    grid_dict['param_grid'] = { field : [value] for (field, value) in params.items() }
    grid_dict['param_grid']['model__seed'] = [i]
    grid_2 = GridSearchCV(NN_2, **grid_dict)
    grid_2.fit(X_train, y_train)

    # save mse and accuracy 
    tr_err.append( grid_2.cv_results_['mean_train_mee'] ) 
    ts_err.append( grid_2.cv_results_['mean_test_mee'] ) 

    # memorize number of epochs (mean over the five folds)
    n_epochs.append(mean([ len(get_histories()[i]['loss']) for i in range(5) ]))

  print("Tr mean mee over 5 inits {:.6f} +/- {:.6f} (std)\n" \
        .format( mean(tr_err), std(tr_err) )
  )

  print("Vl mean mee over 5 inits {:.6f} +/- {:.6f} (std)\n" \
        .format( mean(ts_err), std(ts_err) )
  )

#### Model Assessment

In [ ]:
# retrain on the development set, compute mean error on the internal test set
# and plot learning curve

seed = GRID_DICT['model__seed'][0]
tr_err, tr_acc, ts_err, ts_acc = [], [], [], []
batch_size = kerasRegressorParams['batch_size']

# mean test error and std
for i in range(seed, seed+5):

  NN_params = { field[7:] : value[0] for (field, value) in GRID_DICT.items() }
  NN_params['seed'] = i
  NN = get_NN(len(X_train.columns), **NN_params)

  h = NN.fit(X_train, y_train, batch_size=batch_size, epochs=int(mean(n_epochs)),\
    validation_data=(X_test, y_test), shuffle=True, verbose=0)

  # save mee and accuracy
  tr_score = NN.evaluate(X_train, y_train, verbose=0)
  ts_score = NN.evaluate(X_test, y_test, verbose=0)
  tr_err.append( tr_score ) 
  ts_err.append( ts_score )

# compute mee on test set for a single run
NN_params['seed'] = seed
NN = get_NN(len(X_train.columns), **NN_params)
h = NN.fit(X_train, y_train, batch_size=batch_size, epochs=int(mean(n_epochs)),\
    validation_data=(X_test, y_test), shuffle=True, verbose=0)
tr_single = NN.evaluate(X_train, y_train, verbose=0)
ts_single = NN.evaluate(X_test, y_test, verbose=0)

# print results and plot
print("Tr. mee on a single run: {:.3f} \n".format(tr_single))

print("Tr. mean mee over 5 inits {:.6f} +/- {:.6f} (std)\n".format( mean(tr_err), std(tr_err) ) )

print("Ts. mee on a single run: {:.3f} \n".format(ts_single))

print("Ts. mean mee over 5 inits {:.6f} +/- {:.6f} (std)\n".format( mean(ts_err), std(ts_err) ) )

do_NN_plot(h.history)

In [ ]:
# write CSV blind test output
def write_blind_results(y_pred):

  with open("3Monkeys_ML-CUP23-TS.csv", "w") as f:
    print("# Marco Antonio Corallo, Hamza Karoui, Samuele Vezzuto", file=f)
    print("# 3Monkeys", file=f)
    print("# ML-CUP23", file=f)
    print("# 31/01/2024", file=f)

    pred_id = 1
    for p in y_pred:
      print("{},{},{},{}".format(pred_id, p[0], p[1], p[2]), file=f)
      pred_id += 1

  f.close()

In [ ]:
# retrain also on the test set after the model assessment phase, 
# in order to learn more before to predict blind test output
X, y = pd.concat([X_train, X_test], axis=0), pd.concat([y_train, y_test], axis=0)

NN = get_NN(len(X_train.columns), **NN_params)
NN.fit(X, y, batch_size=batch_size, epochs=int(mean(n_epochs)), shuffle=True, verbose=0)
blind_out = NN.predict(blind_test)
write_blind_results(blind_out)